In [2]:
import pickle as pk
import tensorflow as tf
import numpy as np
import tensorflow as tf
from tensorflow.keras import regularizers

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

import os, sys, pathlib, h5py, time

from astropy.nddata import block_reduce

from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2

%matplotlib inline

2022-08-25 11:23:35.507684: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30130 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0004:04:00.0, compute capability: 7.0
2022-08-25 11:23:35.509887: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30134 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0004:05:00.0, compute capability: 7.0
2022-08-25 11:23:35.512068: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 30134 MB memory:  -> device: 2, name: Tesla V100-SXM2-32GB, pci bus id: 0035:03:00.0, compute capability: 7.0
2022-08-25 11:23:35.514294: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 30134 MB memory:  -> device: 3, name: Tesla V100-SXM2-32GB, pci bus id

In [3]:
data_dir = '/project/r/rbond/jorlo/datasets/ml-clusters/for-debug/'

w_y = 'True'

cut = 1000

In [4]:
pos_im_act = []
pos_im_des = []
neg_im = []

for directory in os.listdir(data_dir):
    print(directory)
    if directory[:3] == 'act' and (int(directory[4:8]) < cut):
        h5f = h5py.File(data_dir+directory)
        pos_im_act.append(h5f['act'][:])
    elif directory[:3] == 'des' and (int(directory[4:8]) < cut):
        h5f = h5py.File(data_dir+directory)
        pos_im_des.append(h5f['des'][:])
    elif directory[:6]=='random' and (int(directory[7:11]) < cut):
        h5f = h5py.File(data_dir+directory)
        neg_im.append(h5f['random'][:])


pos_im_act = np.vstack(pos_im_act)
pos_im_des = np.vstack(pos_im_des)
neg_im = np.vstack(neg_im)


pos_im = np.concatenate((pos_im_act, pos_im_des))


flags = []
for i in range(pos_im.shape[0]):
        if np.any(np.isnan(pos_im[i,...])):
                flags.append(i)

pos_im = np.delete(pos_im, flags, axis = 0)

flags = []
for i in range(neg_im.shape[0]):
        if np.any(np.isnan(neg_im[i,...])):
                flags.append(i)

neg_im = np.delete(neg_im, flags, axis = 0)


neg_im = neg_im[:len(pos_im)]
pos_im = pos_im[:len(neg_im)]

print(len(pos_im), len(neg_im))


random_2000_w_y.h5
des_5000_w_y.h5
des_2000_w_y.h5
random_0000_w_y.h5
des_0000_w_y.h5
des_1000_w_y.h5
random_6000_w_y.h5
random_7000_w_y.h5
random_5000_w_y.h5
des_6000_w_y.h5
act_1000_w_y.h5
random_4000_w_y.h5
random_3000_w_y.h5
act_3000_w_y.h5
des_4000_w_y.h5
random_1000_w_y.h5
act_0000_w_y.h5
des_3000_w_y.h5
922 922


In [13]:
save_dir = '/project/r/rbond/jorlo/datasets/ml-clusters/indv_images/'
with open('/home/r/rbond/jorlo/dev/Keras-FasterRCNN/cluster_data.txt', 'w') as text_f: 
    for i in range(len(pos_im)):
        with h5py.File(save_dir + "pos_img_{}.hdf5".format(i), "w") as f:
            text_f.write('{},100,100,300,300,cluster\n'.format(save_dir + "pos_img_{}.hdf5".format(i)))
            f['data'] = pos_im[i]
    
    for i in range(len(neg_im)):
        with h5py.File(save_dir + "neg_img_{}.hdf5".format(i), "w") as f:
            text_f.write('{},100,100,300,300,bg\n'.format(save_dir + "neg_img_{}.hdf5".format(i)))
            f['data'] = neg_im[i]